## ДЗ2 рейтинг система ЧГК

In [1]:
import re
import pandas as pd
import numpy as np
import pickle
import scipy.stats as stats

from scipy import sparse
from scipy.special import expit

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

## 1. читаем и препроцессим данные

In [2]:
with open('chgk/results.pkl', 'rb') as f1, open('chgk/players.pkl', 'rb') as f2, open('chgk/tournaments.pkl', 'rb') as f3:
    results = pickle.load(f1)
    players = pickle.load(f2)
    tournaments = pickle.load(f3)

In [3]:
tournament_names = {k['id']: k['name'] for i, k in tournaments.items()}
players_names = {k['id']: k['name'] + ' ' + k['surname'] for i, k in players.items()}

# Разделим турниры на трейн и тест по годам
train_tournaments = set([k['id'] for i, k in tournaments.items() if k['dateStart'][:4] == '2019'])
test_tournaments = set([k['id'] for i, k in tournaments.items() if k['dateStart'][:4] == '2020'])

### preprocess

In [4]:
teams_dct = {}
results_train = {}
results_test = {}

for tournament_id, teams_list in results.items():
    tournament_results = {}
    for team in teams_list:
        team_mask = team.get('mask')
        team_members = [player['player']['id'] for player in team['teamMembers']]
        
        # Оставим только команды с игроками
        # И оставим только с бинарной маской ответов
        if team_mask is None or re.findall('[^01]', team_mask) or not team_members:
            continue
          
        team_id = team['team']['id']
        team_name = team['team']['name']
        teams_dct[team_id] = team_name
        
        tournament_results[team_id] = {}
        tournament_results[team_id]['mask'] = team_mask
        tournament_results[team_id]['players'] = team_members
    
    # Уберем турниры с разным числом вопросов +
    #  train/test split
    if len(set(list(map(len, [team['mask'] for team in tournament_results.values()])))) == 1:  
        if tournament_id in train_tournaments:
            results_train[tournament_id] = tournament_results
        elif tournament_id in test_tournaments:
            results_test[tournament_id] = tournament_results

## 2. Baseline

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
1. повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
2. в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
3. для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


Обучаем коэф-ты при игроках и вопросах

 X - OHE-вектор размерности (N, n_questions + n_players)
 
 Создадим трейн выборку

In [5]:
# create train
train = []
max_question_id = 0

for tournament_id, teams in results_train.items():
    for team_id, team in teams.items():
        players = team['players']
        mask = np.array([np.int32(answer) for answer in team['mask']])
        
        questions = np.tile(np.arange(max_question_id,
                                      max_question_id + len(mask)), 
                            len(players))
        
        answers = np.array(np.meshgrid(players, mask)).T.reshape(-1, 2)
        answers = np.hstack([
            np.repeat(tournament_id, len(questions)).reshape(-1, 1),
            np.repeat(team_id, len(questions)).reshape(-1, 1),
            answers, 
            questions.reshape(-1, 1)]
        )
        
        train.append(answers)
        
    max_question_id += len(mask)

In [6]:
train = np.vstack(train).astype(np.int32)
train = pd.DataFrame(train, 
                     columns = ['tournament_id', 
                                'team_id', 
                                'player_id', 
                                'answer', 
                                'question_id'])

In [7]:
# ohe
ohe = OneHotEncoder(handle_unknown='ignore')

X_tr = ohe.fit_transform(train[['player_id', 'question_id']])
y_tr = train['answer']

In [8]:
lr = LogisticRegression(n_jobs=-1, random_state=66)
lr.fit(X_tr, y_tr)

LogisticRegression(n_jobs=-1, random_state=66)

Отранжируем игроков по коэф-ту лог регрессии

In [9]:
unique_players = np.unique(train['player_id'])
unique_questions = np.unique(train['question_id'])
                    
rating = pd.DataFrame({'player_id': unique_players,
                       'strength': lr.coef_[0][:len(unique_players)]})
rating['name'] = rating['player_id'].map(players_names)
rating.sort_values(by='strength', ascending=False).reset_index(drop=True).head(10)

,player_id,strength,name
0,27403,3.790257,Максим Руссо
1,4270,3.537676,Александра Брутер
2,30152,3.388121,Артём Сорожкин
3,37047,3.385634,Мария Юнгер
4,20691,3.299091,Станислав Мереминский
5,28751,3.293894,Иван Семушин
6,27822,3.276289,Михаил Савченков
7,34328,3.238834,Михаил Царёв
8,3843,3.232599,Светлана Бомешко
9,18036,3.225338,Михаил Левандовский


Некоторых игроков я знаю по телевизионному чгк

### 3. Качество рейтинг-системы 
оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. 
**Поэтому:**
1. предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
2. в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Силу команды предлагаю оценивать как вероятность того что хотя бы один член команды ответит на 1 вопрос. При этом сложность вопроса не учитываем

In [10]:
pl_train = set(unique_players)
ques_train = set(unique_questions)

# create test
test = []
for tournament_id, teams in results_test.items():
    for team_id, team in teams.items():
        mask = np.array([np.int32(answer) for answer in team['mask']])
        for player_id in team['players']:
            # оставим только игроков из трейна 
            # наверное лучше подставлять среднюю силу
            if player_id not in pl_train: 
                continue
            
            # -1 - фиктивно добавим вопросы, которых не было
            test.append((tournament_id, team_id, player_id, -1, sum(mask), len(mask))) 
        
test = np.vstack(test).astype(np.int32)
test = pd.DataFrame(test, 
                    columns = ['tournament_id',
                               'team_id',
                               'player_id',
                               'question_id',
                               'n_true',
                               'n_total'])

Transform test and predict

In [11]:
X_test = test[['player_id', 'question_id']]
X_test = ohe.transform(X_test)

preds = lr.predict_proba(X_test)[:, 1]

In [12]:
def compute_scores(data, preds):
    
    helper = lambda x: np.arange(1, len(x)+1)
    
    data['pred'] = preds
    data['score'] = data.groupby([
        'tournament_id',
        'team_id'])['pred'].transform(lambda x: 1 - np.prod(1-x))
    
    rating = data[['tournament_id', 
                   'team_id',
                   'n_true', 
                   'score']].drop_duplicates().reset_index(drop=True)
    
    # Считаем реальный рейтинг команд
    rating = rating.sort_values(by=['tournament_id', 'n_true'], ascending=False)
    rating['real_rank'] = rating.groupby(
        'tournament_id')['n_true'].transform(helper)
    
    # Считаем предсказанный рейтинг
    rating = rating.sort_values(by=['tournament_id', 'score'], ascending=False)
    rating['pred_rank'] = rating.groupby('tournament_id')['score'].transform(helper)

    rating = rating.astype(np.int32)
    
    spearman_corr = rating.groupby('tournament_id').apply(lambda x: 
                                                          stats.spearmanr(
                                                              x['real_rank'],
                                                              x['pred_rank']).correlation).mean()
    
    kendal_corr = rating.groupby('tournament_id').apply(lambda x: 
                                                        stats.kendalltau(
                                                            x['real_rank'],
                                                            x['pred_rank']).correlation).mean()
    
    print(f"Корреляция Спирмана: {spearman_corr}")

In [13]:
compute_scores(test, preds)

Корреляция Спирмана: 0.772581765976506


#### 4. Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
1. предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
2. разработайте EM-схему для обучения этой модели, реализуйте её в коде;
3. обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


 
До этого мы считали, что если команда ответила на вопрос, то и игрок на него ответил. На самом деле это не так, и нам нужно оценить вероятность ответа игрока при условии "силы команды". "Силой команды" можно назвать, например, среднее число ответивших на вопрос игроков, но там образуются сложные вычисления с binominal poisson distribution. Предлагаю попробовать считать "силой команды", как и ранее, вероятность хотя бы одного игрока ответить на вопрос. Также предположим, что "сила команды", если игрок ответил верно, равна 1. Тогда:

    1. P(player = 1 | team) = P(team | player = 1) * P(player = 1) / P(team) = P(player = 1) / P(team)
    2. Также будем считать, что P(player = 1 | team ) = 0, если команда ответила на вопрос неверно

Соответсвенно, на E-шаге оцениваем P(player = 1 | team), а на M-шаге обучаем логистическую регрессию на этом таргете, на выходе получаем значения P(player = 1).

P(team), как и ранее, оцениваем как вероятность того, что хотя бы 1 игрок ответит верно: P(team) = 1 - П[1 - P(player = 1)]


In [16]:
def log_loss(y_true, y_pred):
    return - np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

class MyEMClassifier:
    def __init__(self, w=None, lr=23, n_iter=30, batch_size=5000, verbose=1):
        self.w = w
        self.lr = lr
        self.n_iter = n_iter
        self.batch_size = batch_size
        self.verbose = 1
        
    def _add_intercept(self, X):
        return sparse.hstack((np.ones((X.shape[0], 1)), X), format='csr')
    
    def _init_w(self, dim):
        self.w = np.random.randn(dim)
        
    def _E_step(self, data, preds):
        team_strength = pd.DataFrame({'team_id': data['team_id'],
                                      'question_id': data['question_id'], 
                                      'team_strength': 1 - preds})
        team_strength = team_strength.groupby(['team_id', 
                                               'question_id']).agg({'team_strength': 'prod'}).reset_index()
        
        team_strength['team_strength'] = 1 - team_strength['team_strength']
        team_strength = data[['team_id', 'question_id']].merge(team_strength)
        # переведем к вероятностям
        y = np.clip(preds / team_strength['team_strength'], 0, 1).values 
        y[data['answer'] == 0] = 0
        return y
        
    def _M_step(self, X, y):
        # Обучаем LogReg батчевым градиентным спуском
        min_loss = np.inf
        indices = np.arange(X.shape[0])
        for _ in range(100):
            indices = np.random.permutation(indices)
            for batch_idx in np.array_split(indices, len(indices) // self.batch_size):
                x_batch, y_batch = X[batch_idx], y[batch_idx]
                grad = x_batch.T.dot(self.predict(x_batch) - y_batch) / len(y_batch)
                self.w -= self.lr * grad
                
            cur_loss = log_loss(y, self.predict(X))
            if min_loss - cur_loss < 1e-6:
                break
                
            min_loss = cur_loss
                
    def fit(self, X_tr, train_data, X_te=None, test_data=None):
        X_tr = self._add_intercept(X_tr)
        if self.w is None or len(self.w) != X_tr.shape[1]:
            self._init_w(X_tr.shape[1])
        
        for iter_ in tqdm(range(self.n_iter)): 
            preds = self.predict(X_tr)
            y = self._E_step(train_data, preds)
            self._M_step(X_tr, y)
            if self.verbose is not None and X_te is not None and test_data is not None and iter_ % self.verbose == 0:
                compute_scores(test_data, self.predict(X_te))
                         
    def predict(self, X):
        if self.w is None:
            raise ValueError('Model is not fitted yet!')
        if len(self.w) != X.shape[1]:
            X = self._add_intercept(X)
        return expit(X.dot(self.w))

In [17]:
# Инициализируем нашей обученной моделькой, чтобы не ждать вечность
w_init = np.hstack([lr.intercept_, lr.coef_[0]])
em_classifier = MyEMClassifier(w_init, n_iter=10)

# Обучим 10 эпох
em_classifier.fit(X_tr, train, X_test, test)

 10%|█         | 1/10 [01:45<15:45, 105.03s/it]

Корреляция Спирмана: 0.7816405458699732


 20%|██        | 2/10 [04:06<15:28, 116.10s/it]

Корреляция Спирмана: 0.7879400487566784


 30%|███       | 3/10 [05:35<12:35, 107.88s/it]

...

Корреляция Спирмана: 0.7866519965829105


 90%|█████████ | 9/10 [14:47<01:33, 93.84s/it]

Корреляция Спирмана: 0.7865623777248353


100%|██████████| 10/10 [16:18<00:00, 97.83s/it]

Корреляция Спирмана: 0.786003093817423


In [18]:
rating = pd.DataFrame({'player_id': unique_players,
                       'strength': em_classifier.w[1:1 + len(unique_players)]})
rating['name'] = rating['player_id'].map(players_names)
rating['questions_count'] = rating['player_id'].map(train.groupby('player_id')['question_id'].count())

In [19]:
rating.sort_values(by='strength', ascending=False).head(50)

,player_id,strength,name,questions_count
4152,30260,3.734190,Евгений Спектор,233
3767,27403,3.612572,Максим Руссо,1796
5234,38196,3.576408,Артём Митрофанов,546
2087,15123,3.550754,Ирина Колесникова,441
3546,25757,3.540146,Мария Летюхина,291
7961,74001,3.530864,Игорь Мокин,900
4886,35877,3.524439,Богдан Шевченко,641
4477,32765,3.452747,Полина Усыскин,646
5054,37047,3.425230,Мария Юнгер,452
1251,9061,3.381557,Евгений Дёмин,369


#### 5. А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).


Сложность турнира посчитаем как среднюю сложность вопроса - возьмем средние коэффициенты нашей модели.


In [20]:
q_rating = dict(zip(unique_questions, em_classifier.w[-len(unique_questions):]))

train['difficulty'] = train['question_id'].map(q_rating)
train['tournament_name'] = train['tournament_id'].map(tournament_names)


In [21]:
tournaments_rating = train[['tournament_name', 'question_id', 'difficulty']].drop_duplicates()
tournaments_rating = tournaments_rating.groupby('tournament_name')['difficulty'].mean().sort_values().reset_index()

In [22]:
# самые сложные вопросы 
# сверху вниз
tournaments_rating.head(20)

,tournament_name,difficulty
0,Чемпионат Санкт-Петербурга. Первая лига,-3.897668
1,Угрюмый Ёрш,-2.138863
2,Первенство правого полушария,-1.841768
3,Воображаемый музей,-1.647435
4,Кубок городов,-1.644300
5,Записки охотника,-1.519809
6,Чемпионат России,-1.456632
7,Ускользающая сова,-1.448631
8,Чемпионат Мира. Этап 3. Группа В,-1.442103
9,All Cats Are Beautiful,-1.426012


In [23]:
# Самые простые турниры снизу вверх
tournaments_rating.tail(10)

,tournament_name,difficulty
588,Студенческий чемпионат Калининградской области,1.429099
589,Joystick Cup,1.438572
590,(а)Синхрон-lite. Лига старта. Эпизод X,1.471189
591,(а)Синхрон-lite. Лига старта. Эпизод IV,1.501554
592,Второй тематический турнир имени Джоуи Триббиани,1.517323
593,(а)Синхрон-lite. Лига старта. Эпизод VI,1.609064
594,(а)Синхрон-lite. Лига старта. Эпизод VII,1.637814
595,(а)Синхрон-lite. Лига старта. Эпизод III,1.766532
596,(а)Синхрон-lite. Лига старта. Эпизод IX,1.771884
597,(а)Синхрон-lite. Лига старта. Эпизод V,1.877995
